# 🤖 Modern Robotics: 理論から実装まで
## Northwestern University専門講座の統合インタラクティブノートブック

---

### 📚 講座の全体像
このノートブックは、**6つのコースで構成される包括的なロボティクス教育**を統合した学習体験を提供します：

1. **Course 1**: Configuration Space（C空間）- ロボットの「居場所」を数学的に定義
2. **Course 2**: Forward/Inverse Kinematics - 関節角度と位置の相互変換
3. **Course 3**: Robot Dynamics - 力とトルクの世界
4. **Course 4**: Trajectory Generation - 軌道生成と時間最適制御
5. **Course 5**: Grasping & Mobile Robots - 把持とモバイルロボティクス
6. **Course 6**: Capstone Project - KUKA youBotによる統合実装

### 🎯 なぜModern Roboticsなのか？

**従来のアプローチ vs Modern Robotics:**
- **従来**: D-H表現、複雑な座標変換
- **Modern**: Lie群論、Product of Exponentials、統一された数学的枠組み

**統一理論の威力:**
$$SE(3) = \{T \in \mathbb{R}^{4×4} : T = \begin{bmatrix} R & p \\ 0 & 1 \end{bmatrix}, R \in SO(3), p \in \mathbb{R}^3\}$$

この数式一つで、すべての剛体変換を表現できます。

---

### 🔧 使用方法
1. 各セクションのコードセルを順番に実行
2. インタラクティブなウィジェットで実験
3. パラメータを変更して結果を観察
4. 理論と実装の両面から学習

## 📦 1. Environment Setup and Imports

このセクションでは、ノートブック全体で必要なライブラリをインポートし、インタラクティブプロット環境を設定します。

In [ ]:
# Essential imports for Modern Robotics
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle, Ellipse, Rectangle, FancyArrowPatch
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D, proj3d
import warnings
warnings.filterwarnings('ignore')

# Interactive widgets
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
    print("✅ Interactive widgets available")
except ImportError:
    WIDGETS_AVAILABLE = False
    print("⚠️ ipywidgets not available - static plots only")

# Modern Robotics library (install if needed)
try:
    import modern_robotics as mr
    print("✅ Modern Robotics library loaded")
except ImportError:
    print("⚠️ Installing modern_robotics library...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'modern_robotics'])
    import modern_robotics as mr
    print("✅ Modern Robotics library installed and loaded")

# Configure matplotlib for inline plotting
%matplotlib inline
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Enable JavaScript for interactive plots if available
try:
    %matplotlib widget
    print("✅ Interactive matplotlib widgets enabled")
except:
    print("ℹ️ Using inline matplotlib")

print("🔧 Environment setup complete!")
print("📊 Numpy version:", np.__version__)
print("📈 Matplotlib version:", plt.matplotlib.__version__)

## 📖 2. Course Overview and Mathematical Foundations

### 数学的基礎：Lie群とスクリュー理論

Modern Roboticsの美しさは、複雑な3次元変換を**統一された数学的枠組み**で扱えることです。

In [ ]:
# Define utility functions for the entire notebook
def skew_symmetric(v):
    """Convert 3D vector to skew-symmetric matrix"""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def rotation_matrix_rodrigues(axis, theta):
    """Create rotation matrix using Rodrigues' formula"""
    axis = axis / np.linalg.norm(axis)
    K = skew_symmetric(axis)
    return np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * K @ K

def homogeneous_transform(R, p):
    """Create 4x4 homogeneous transformation matrix"""
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = p
    return T

# Display mathematical foundations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# SO(3) - Rotation group
ax1 = axes[0]
ax1.text(0.5, 0.7, r'$SO(3)$', fontsize=20, ha='center', transform=ax1.transAxes)
ax1.text(0.5, 0.5, r'$R^T R = I$', fontsize=12, ha='center', transform=ax1.transAxes)
ax1.text(0.5, 0.4, r'$\det(R) = 1$', fontsize=12, ha='center', transform=ax1.transAxes)
ax1.text(0.5, 0.2, '3D Rotations', fontsize=14, ha='center', transform=ax1.transAxes, weight='bold')
ax1.set_title('Special Orthogonal Group')
ax1.axis('off')

# SE(3) - Euclidean group
ax2 = axes[1]
ax2.text(0.5, 0.7, r'$SE(3)$', fontsize=20, ha='center', transform=ax2.transAxes)
ax2.text(0.5, 0.5, r'$T = \begin{bmatrix} R & p \\ 0 & 1 \end{bmatrix}$', fontsize=12, ha='center', transform=ax2.transAxes)
ax2.text(0.5, 0.2, 'Rigid Transformations', fontsize=14, ha='center', transform=ax2.transAxes, weight='bold')
ax2.set_title('Special Euclidean Group')
ax2.axis('off')

# Lie algebra se(3)
ax3 = axes[2]
ax3.text(0.5, 0.7, r'$se(3)$', fontsize=20, ha='center', transform=ax3.transAxes)
ax3.text(0.5, 0.5, r'$[\omega, v] = \begin{bmatrix} [\omega]_× & v \\ 0 & 0 \end{bmatrix}$', fontsize=12, ha='center', transform=ax3.transAxes)
ax3.text(0.5, 0.2, 'Screw Motions', fontsize=14, ha='center', transform=ax3.transAxes, weight='bold')
ax3.set_title('Lie Algebra')
ax3.axis('off')

plt.tight_layout()
plt.show()

# Product of Exponentials formula
print("🔑 Key Formula - Product of Exponentials:")
print("T(θ) = e^{[S₁]θ₁} e^{[S₂]θ₂} ... e^{[Sₙ]θₙ} M")
print("\nWhere:")
print("• Sᵢ = screw axis of joint i")
print("• θᵢ = joint variable i")
print("• M = home configuration")
print("• e^{[S]θ} = matrix exponential (screw motion)")
print("\n✨ This single equation handles ANY robot geometry!")

## 🌍 3. Configuration Space Interactive Demo

### Course 1: Configuration Space（C空間）

**コンセプト**: ロボットの「居場所」を数学的に定義する空間

**重要な洞察**: 関節空間での直線は、作業空間では曲線になる！

In [ ]:
# Configuration Space Demo - 2R Planar Arm
class TwoLinkArm:
    def __init__(self, L1=1.0, L2=0.7):
        self.L1 = L1  # Length of first link
        self.L2 = L2  # Length of second link
        
    def forward_kinematics(self, theta1, theta2):
        """Compute end-effector position from joint angles"""
        x1 = self.L1 * np.cos(theta1)
        y1 = self.L1 * np.sin(theta1)
        
        x2 = x1 + self.L2 * np.cos(theta1 + theta2)
        y2 = y1 + self.L2 * np.sin(theta1 + theta2)
        
        return x1, y1, x2, y2
    
    def get_workspace_boundary(self, n_points=1000):
        """Get the reachable workspace boundary"""
        theta = np.linspace(0, 2*np.pi, n_points)
        
        # Outer circle (full extension)
        r_max = self.L1 + self.L2
        outer_x = r_max * np.cos(theta)
        outer_y = r_max * np.sin(theta)
        
        # Inner circle (if L1 > L2)
        if self.L1 > self.L2:
            r_min = self.L1 - self.L2
            inner_x = r_min * np.cos(theta)
            inner_y = r_min * np.sin(theta)
        else:
            inner_x = inner_y = None
            
        return outer_x, outer_y, inner_x, inner_y

def plot_configuration_space_demo():
    """Static demonstration of configuration space concepts"""
    robot = TwoLinkArm(L1=1.0, L2=0.7)
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Configuration space (left)
    ax1.set_title('Configuration Space (Joint Space)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('θ₁ (rad)', fontsize=12)
    ax1.set_ylabel('θ₂ (rad)', fontsize=12)
    ax1.set_xlim([0, 2*np.pi])
    ax1.set_ylim([0, 2*np.pi])
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    
    # Task space (right)
    ax2.set_title('Task Space (Workspace)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('x (m)', fontsize=12)
    ax2.set_ylabel('y (m)', fontsize=12)
    ax2.set_xlim([-2, 2])
    ax2.set_ylim([-2, 2])
    ax2.grid(True, alpha=0.3)
    ax2.set_aspect('equal')
    
    # Draw workspace boundary
    outer_x, outer_y, inner_x, inner_y = robot.get_workspace_boundary()
    ax2.plot(outer_x, outer_y, 'k--', alpha=0.5, label='Workspace boundary')
    if inner_x is not None:
        ax2.plot(inner_x, inner_y, 'k--', alpha=0.5)
    
    # Demonstrate straight line in joint space -> curved path in task space
    n_points = 20
    t = np.linspace(0, 1, n_points)
    
    # Path 1: Diagonal line in joint space
    theta1_path = np.pi/6 + t * (2*np.pi/3)
    theta2_path = np.pi/3 + t * (2*np.pi/3)
    
    # Plot joint space path
    ax1.plot(theta1_path, theta2_path, 'b-', linewidth=3, label='Straight line path', marker='o', markersize=4)
    ax1.plot(theta1_path[0], theta2_path[0], 'go', markersize=12, label='Start')
    ax1.plot(theta1_path[-1], theta2_path[-1], 'ro', markersize=12, label='End')
    
    # Compute task space path
    x_path, y_path = [], []
    for theta1, theta2 in zip(theta1_path, theta2_path):
        _, _, x, y = robot.forward_kinematics(theta1, theta2)
        x_path.append(x)
        y_path.append(y)
    
    # Plot task space path
    ax2.plot(x_path, y_path, 'b-', linewidth=3, label='Resulting curved path', marker='o', markersize=4)
    ax2.plot(x_path[0], y_path[0], 'go', markersize=12, label='Start')
    ax2.plot(x_path[-1], y_path[-1], 'ro', markersize=12, label='End')
    
    # Draw current robot configuration (middle point)
    mid_idx = len(theta1_path) // 2
    theta1_mid, theta2_mid = theta1_path[mid_idx], theta2_path[mid_idx]
    x1, y1, x2, y2 = robot.forward_kinematics(theta1_mid, theta2_mid)
    
    # Robot visualization
    ax2.plot([0, x1], [0, y1], 'k-', linewidth=4, alpha=0.7)
    ax2.plot([x1, x2], [y1, y2], 'k-', linewidth=4, alpha=0.7)
    ax2.plot(0, 0, 'ko', markersize=8)
    ax2.plot(x1, y1, 'ko', markersize=6)
    ax2.plot(x2, y2, 'ro', markersize=8)
    
    # Add topology information
    ax1.text(np.pi, -0.3, '↔ Periodic (wraps at 2π)', ha='center', fontsize=10, color='blue')
    ax1.text(-0.3, np.pi, '↕ Periodic', ha='center', rotation=90, fontsize=10, color='blue')
    
    ax1.legend()
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    return fig

# Run the demonstration
print("🌍 Configuration Space Demonstration")
print("=" * 40)
fig = plot_configuration_space_demo()

print("\n💡 Key Insights:")
print("• Configuration space has torus topology: T² = S¹ × S¹")
print("• Each joint angle wraps at 2π (periodic boundaries)")
print("• Straight lines in C-space → curved paths in task space")
print("• This mapping is the essence of robot kinematics!")

In [ ]:
# Interactive Configuration Space Widget (if widgets available)
if WIDGETS_AVAILABLE:
    def interactive_2r_arm(theta1=np.pi/4, theta2=np.pi/3):
        """Interactive 2R arm visualization"""
        robot = TwoLinkArm()
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Configuration space
        ax1.set_title('Configuration Space')
        ax1.set_xlabel('θ₁ (rad)')
        ax1.set_ylabel('θ₂ (rad)')
        ax1.set_xlim([0, 2*np.pi])
        ax1.set_ylim([0, 2*np.pi])
        ax1.grid(True, alpha=0.3)
        
        # Mark current point
        ax1.plot(theta1, theta2, 'ro', markersize=10, label=f'Current: ({theta1:.2f}, {theta2:.2f})')
        
        # Task space
        ax2.set_title('Task Space')
        ax2.set_xlabel('x (m)')
        ax2.set_ylabel('y (m)')
        ax2.set_xlim([-2, 2])
        ax2.set_ylim([-2, 2])
        ax2.grid(True, alpha=0.3)
        ax2.set_aspect('equal')
        
        # Draw workspace
        outer_x, outer_y, inner_x, inner_y = robot.get_workspace_boundary()
        ax2.plot(outer_x, outer_y, 'k--', alpha=0.3)
        if inner_x is not None:
            ax2.plot(inner_x, inner_y, 'k--', alpha=0.3)
        
        # Draw robot
        x1, y1, x2, y2 = robot.forward_kinematics(theta1, theta2)
        ax2.plot([0, x1], [0, y1], 'k-', linewidth=4)
        ax2.plot([x1, x2], [y1, y2], 'k-', linewidth=4)
        ax2.plot(0, 0, 'ko', markersize=8)
        ax2.plot(x1, y1, 'ko', markersize=6)
        ax2.plot(x2, y2, 'ro', markersize=8, label=f'End-effector: ({x2:.2f}, {y2:.2f})')
        
        ax1.legend()
        ax2.legend()
        plt.tight_layout()
        plt.show()
    
    # Create interactive widget
    theta1_slider = widgets.FloatSlider(value=np.pi/4, min=0, max=2*np.pi, step=0.1, description='θ₁:')
    theta2_slider = widgets.FloatSlider(value=np.pi/3, min=0, max=2*np.pi, step=0.1, description='θ₂:')
    
    interactive_widget = widgets.interactive(interactive_2r_arm, theta1=theta1_slider, theta2=theta2_slider)
    display(interactive_widget)
    
    print("🎮 Use the sliders above to explore the configuration space!")
    print("Notice how joint angles map to end-effector positions.")
else:
    print("⚠️ Interactive widgets not available. Install ipywidgets for full experience.")
    print("pip install ipywidgets")
    print("jupyter nbextension enable --py widgetsnbextension")

## 🔄 4. Rigid Body Motion and SE(3) Visualization

### Course 2: Rigid Body Motion - SE(3) and Twist Visualization

**統一理論の美しさ**: 回転と並進を一つの数学的枠組みで扱う

- **SO(3)**: 回転群（3×3直交行列、det=1）
- **SE(3)**: 特殊ユークリッド群（4×4同次変換行列）
- **Screw Motion**: 軸周りの回転と軸方向の並進の組み合わせ

In [ ]:
# 3D Arrow class for visualization
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        return np.min(zs)

def se3_to_matrix(omega, v, theta):
    """Convert screw parameters to SE(3) matrix using exponential map"""
    if np.linalg.norm(omega) < 1e-6:
        # Pure translation
        R = np.eye(3)
        p = v * theta
    else:
        omega_hat = omega / np.linalg.norm(omega)
        R = rotation_matrix_rodrigues(omega_hat, theta)
        
        # Compute translation using screw motion formula
        omega_skew = skew_symmetric(omega_hat)
        G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * omega_skew + \
                  (theta - np.sin(theta)) * omega_skew @ omega_skew
        p = G_theta @ v
    
    return homogeneous_transform(R, p)

def draw_frame_3d(ax, T, label='', scale=0.5, alpha=1.0):
    """Draw a coordinate frame at pose T"""
    origin = T[:3, 3]
    R = T[:3, :3]
    
    # Draw axes
    colors = ['r', 'g', 'b']
    for i, color in enumerate(colors):
        direction = R[:, i] * scale
        arrow = Arrow3D([origin[0], origin[0] + direction[0]],
                       [origin[1], origin[1] + direction[1]],
                       [origin[2], origin[2] + direction[2]],
                       mutation_scale=20, lw=2, arrowstyle='-|>',
                       color=color, alpha=alpha)
        ax.add_artist(arrow)
    
    if label:
        ax.text(origin[0], origin[1], origin[2], label, fontsize=10)

def demonstrate_rigid_body_motion():
    """Demonstrate SO(3) and SE(3) concepts"""
    fig = plt.figure(figsize=(16, 8))
    
    # SO(3) demonstration
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.set_title('SO(3): Rotation Matrix Evolution', fontsize=14, fontweight='bold')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_xlim([-1.5, 1.5])
    ax1.set_ylim([-1.5, 1.5])
    ax1.set_zlim([-1.5, 1.5])
    
    # SE(3) demonstration
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.set_title('SE(3): Screw Motion', fontsize=14, fontweight='bold')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    ax2.set_xlim([-2, 2])
    ax2.set_ylim([-2, 2])
    ax2.set_zlim([0, 4])
    
    # Show rotation sequence
    axis_so3 = np.array([1, 1, 1]) / np.sqrt(3)  # Normalized axis
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi]
    
    for i, theta in enumerate(angles):
        R = rotation_matrix_rodrigues(axis_so3, theta)
        T = homogeneous_transform(R, np.zeros(3))
        alpha = 0.3 + 0.7 * (i / (len(angles) - 1))
        draw_frame_3d(ax1, T, f't={i}', scale=0.8, alpha=alpha)
    
    # Draw rotation axis
    ax1.plot([0, axis_so3[0]], [0, axis_so3[1]], [0, axis_so3[2]], 'k--', alpha=0.5, linewidth=2)
    ax1.text(axis_so3[0]*1.2, axis_so3[1]*1.2, axis_so3[2]*1.2, 'rotation axis', fontsize=8)
    
    # Show screw motion sequence
    omega = np.array([0, 0, 1])  # Rotation around z-axis
    v = np.array([0, 0, 0.5])    # Translation along z-axis (pitch)
    
    trajectory_points = []
    for i, theta in enumerate(angles):
        T_screw = se3_to_matrix(omega, v, theta)
        alpha = 0.3 + 0.7 * (i / (len(angles) - 1))
        draw_frame_3d(ax2, T_screw, f't={i}', scale=0.5, alpha=alpha)
        trajectory_points.append(T_screw[:3, 3])
    
    # Draw screw trajectory
    if len(trajectory_points) > 1:
        traj = np.array(trajectory_points)
        ax2.plot(traj[:, 0], traj[:, 1], traj[:, 2], 'b-', linewidth=2, alpha=0.7, label='Screw trajectory')
    
    # Draw screw axis
    ax2.plot([0, 0], [0, 0], [0, 3], 'k--', alpha=0.5, linewidth=2)
    ax2.text(0.1, 0.1, 3, 'screw axis', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    return fig

# Demonstrate the concepts
print("🔄 Rigid Body Motion Demonstration")
print("=" * 40)
fig = demonstrate_rigid_body_motion()

# Mathematical explanation
print("\\n📐 Mathematical Foundation:")
print("• Rodrigues' Formula: R = e^{[ω̂]θ} = I + sin(θ)[ω̂] + (1-cos(θ))[ω̂]²")
print("• Matrix Exponential: T = e^{[S]θ} where S = (ω, v) is the screw axis")
print("• Screw Motion: Combined rotation + translation along/about same axis")

print("\\n💡 Key Insights:")
print("• SO(3): Pure rotations in 3D space")
print("• SE(3): Rigid body transformations (rotation + translation)")
print("• Screw theory unifies all rigid body motions")
print("• Matrix exponential provides the connection between Lie algebra and Lie group")

## ⚙️ 5. Forward Kinematics with Product of Exponentials

### Course 2: 順運動学 - Product of Exponentials (PoE) アプローチ

**革新的アプローチ**: 各関節を空間内のスクリュー軸として扱う

**PoE公式**: T(θ) = e^{[S₁]θ₁} e^{[S₂]θ₂} ... e^{[Sₙ]θₙ} M

- **Sᵢ**: 関節iのスクリュー軸
- **θᵢ**: 関節変数i  
- **M**: ホーム構成
- **e^{[S]θ}**: 行列指数（スクリュー運動）

In [ ]:
# Forward Kinematics using Product of Exponentials
def exp_twist(S, theta):
    """Matrix exponential for twist"""
    omega = S[:3]
    v = S[3:]
    
    if np.linalg.norm(omega) < 1e-6:
        # Pure translation
        T = np.eye(4)
        T[:3, 3] = v * theta
        return T
    
    # Rotation with translation (screw motion)
    omega_skew = skew_symmetric(omega)
    omega_norm = np.linalg.norm(omega)
    omega_hat = omega / omega_norm
    omega_hat_skew = skew_symmetric(omega_hat)
    
    theta_norm = theta * omega_norm
    
    # Rodrigues' formula for rotation
    R = np.eye(3) + np.sin(theta_norm) * omega_hat_skew + \
        (1 - np.cos(theta_norm)) * omega_hat_skew @ omega_hat_skew
    
    # Translation component
    G = np.eye(3) * theta_norm + (1 - np.cos(theta_norm)) * omega_hat_skew + \
        (theta_norm - np.sin(theta_norm)) * omega_hat_skew @ omega_hat_skew
    p = G @ (v / omega_norm)
    
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = p
    
    return T

class ThreeJointRobot:
    """3R planar robot for PoE demonstration"""
    
    def __init__(self):
        # Link lengths
        self.L1 = 1.0
        self.L2 = 0.8
        self.L3 = 0.6
        
        # Home configuration (all joints at 0)
        self.M = np.array([
            [1, 0, 0, self.L1 + self.L2 + self.L3],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        
        # Screw axes at home configuration (all rotate around z-axis)
        self.S1 = np.array([0, 0, 1, 0, 0, 0])  # Joint 1 at origin
        
        # Joint 2 at (L1, 0, 0)
        q2 = np.array([self.L1, 0, 0])
        omega2 = np.array([0, 0, 1])
        v2 = -np.cross(omega2, q2)
        self.S2 = np.concatenate([omega2, v2])
        
        # Joint 3 at (L1+L2, 0, 0)
        q3 = np.array([self.L1 + self.L2, 0, 0])
        omega3 = np.array([0, 0, 1])
        v3 = -np.cross(omega3, q3)
        self.S3 = np.concatenate([omega3, v3])
        
    def forward_kinematics_poe(self, theta):
        """Compute forward kinematics using PoE"""
        # Compute exponentials
        exp_S1_theta1 = exp_twist(self.S1, theta[0])
        exp_S2_theta2 = exp_twist(self.S2, theta[1])
        exp_S3_theta3 = exp_twist(self.S3, theta[2])
        
        # Product of exponentials
        T = exp_S1_theta1 @ exp_S2_theta2 @ exp_S3_theta3 @ self.M
        
        # Also compute intermediate joint positions
        T1 = exp_S1_theta1 @ homogeneous_transform(np.eye(3), np.array([self.L1, 0, 0]))
        T2 = exp_S1_theta1 @ exp_S2_theta2 @ homogeneous_transform(np.eye(3), np.array([self.L1 + self.L2, 0, 0]))
        
        return T, T1, T2
    
    def get_link_positions(self, theta):
        """Get all joint and end-effector positions"""
        T, T1, T2 = self.forward_kinematics_poe(theta)
        
        positions = {
            'base': np.array([0, 0]),
            'joint1': T1[:2, 3],
            'joint2': T2[:2, 3],
            'end': T[:2, 3]
        }
        
        return positions

def demonstrate_forward_kinematics():
    """Demonstrate Product of Exponentials forward kinematics"""
    robot = ThreeJointRobot()
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Robot workspace visualization
    ax1.set_title('Robot Workspace', fontsize=14, fontweight='bold')
    ax1.set_xlabel('x (m)')
    ax1.set_ylabel('y (m)')
    ax1.set_xlim([-3, 3])
    ax1.set_ylim([-3, 3])
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    
    # Screw axes at home configuration
    ax2.set_title('Screw Axes at Home Configuration', fontsize=14, fontweight='bold')
    ax2.set_xlabel('x (m)')
    ax2.set_ylabel('y (m)')
    ax2.set_xlim([-0.5, 3])
    ax2.set_ylim([-1.5, 1.5])
    ax2.set_aspect('equal')
    ax2.grid(True, alpha=0.3)
    
    # Draw home configuration
    ax2.plot([0, robot.L1], [0, 0], 'k-', linewidth=3, alpha=0.5, label='Home config')
    ax2.plot([robot.L1, robot.L1 + robot.L2], [0, 0], 'k-', linewidth=3, alpha=0.5)
    ax2.plot([robot.L1 + robot.L2, robot.L1 + robot.L2 + robot.L3], [0, 0], 'k-', linewidth=3, alpha=0.5)
    
    # Draw screw axes
    ax2.arrow(0, 0, 0, 0.3, head_width=0.05, head_length=0.05, fc='red', ec='red')
    ax2.text(0, 0.4, 'S₁', fontsize=12, ha='center', color='red', fontweight='bold')
    
    ax2.arrow(robot.L1, 0, 0, 0.3, head_width=0.05, head_length=0.05, fc='green', ec='green')
    ax2.text(robot.L1, 0.4, 'S₂', fontsize=12, ha='center', color='green', fontweight='bold')
    
    ax2.arrow(robot.L1 + robot.L2, 0, 0, 0.3, head_width=0.05, head_length=0.05, fc='blue', ec='blue')
    ax2.text(robot.L1 + robot.L2, 0.4, 'S₃', fontsize=12, ha='center', color='blue', fontweight='bold')
    
    ax2.legend()
    
    # PoE build-up visualization
    ax3.set_title('Product of Exponentials Build-up', fontsize=14, fontweight='bold')
    ax3.text(0.5, 0.8, r'$T = e^{[S_1]θ_1} \cdot e^{[S_2]θ_2} \cdot e^{[S_3]θ_3} \cdot M$', 
             fontsize=14, ha='center', transform=ax3.transAxes)
    ax3.text(0.5, 0.6, 'Forward Kinematics via Matrix Exponentials', 
             fontsize=12, ha='center', transform=ax3.transAxes, fontweight='bold')
    ax3.text(0.5, 0.4, '• Each joint contributes a screw motion', 
             fontsize=11, ha='center', transform=ax3.transAxes)
    ax3.text(0.5, 0.3, '• Composed via matrix multiplication', 
             fontsize=11, ha='center', transform=ax3.transAxes)
    ax3.text(0.5, 0.2, '• No D-H parameters needed!', 
             fontsize=11, ha='center', transform=ax3.transAxes, style='italic', color='blue')
    ax3.axis('off')
    
    # End-effector trajectory
    ax4.set_title('End-Effector Trajectories', fontsize=14, fontweight='bold')
    ax4.set_xlabel('x (m)')
    ax4.set_ylabel('y (m)')
    ax4.set_xlim([-3, 3])
    ax4.set_ylim([-3, 3])
    ax4.set_aspect('equal')
    ax4.grid(True, alpha=0.3)
    
    # Show different robot configurations
    configs = [
        [0, 0, 0],           # Home
        [np.pi/4, 0, 0],     # Joint 1 only
        [np.pi/4, np.pi/3, 0], # Joints 1&2
        [np.pi/4, np.pi/3, -np.pi/4] # All joints
    ]
    
    colors = ['black', 'red', 'green', 'blue']
    labels = ['Home', 'θ₁ only', 'θ₁ + θ₂', 'All joints']
    
    for i, (theta, color, label) in enumerate(zip(configs, colors, labels)):
        positions = robot.get_link_positions(theta)
        
        # Draw robot in workspace
        ax1.plot([positions['base'][0], positions['joint1'][0]], 
                [positions['base'][1], positions['joint1'][1]], 
                color=color, linewidth=3, alpha=0.7)
        ax1.plot([positions['joint1'][0], positions['joint2'][0]], 
                [positions['joint1'][1], positions['joint2'][1]], 
                color=color, linewidth=2.5, alpha=0.7)
        ax1.plot([positions['joint2'][0], positions['end'][0]], 
                [positions['joint2'][1], positions['end'][1]], 
                color=color, linewidth=2, alpha=0.7, label=label)
        
        # Mark end-effector
        ax1.plot(positions['end'][0], positions['end'][1], 'o', 
                color=color, markersize=8)
        
        # Show trajectory in separate plot
        ax4.plot(positions['end'][0], positions['end'][1], 'o', 
                color=color, markersize=10, label=f"{label}: ({positions['end'][0]:.2f}, {positions['end'][1]:.2f})")
    
    ax1.plot(0, 0, 'ko', markersize=10, label='Base')
    ax1.legend()
    ax4.legend()
    
    plt.tight_layout()
    plt.show()
    
    return fig

# Demonstrate forward kinematics
print("⚙️ Forward Kinematics - Product of Exponentials")
print("=" * 50)
fig = demonstrate_forward_kinematics()

# Screw axis information
robot = ThreeJointRobot()
print("\\n📊 Screw Axes Definition:")
print(f"S₁ = {robot.S1} (Joint 1)")
print(f"S₂ = {robot.S2} (Joint 2)")  
print(f"S₃ = {robot.S3} (Joint 3)")

print("\\n💡 Key Advantages of PoE:")
print("• Geometric intuition: each joint is a screw axis")
print("• No singular representations (unlike Euler angles)")
print("• Unified approach for any robot geometry")
print("• Direct connection to Lie group theory")
print("• Computationally efficient for modern algorithms")

## 📐 6. Jacobian Analysis and Manipulability Visualization

### ヤコビアンと速度運動学

**基本方程式**: V = J(θ) θ̇

- **V**: エンドエフェクタの空間速度（twist）
- **J(θ)**: ヤコビアン行列（構成依存）
- **θ̇**: 関節速度ベクトル

**重要概念**:
- **Manipulability Ellipsoid**: ロボットの動作能力を可視化
- **Condition Number**: 特異点への近さを定量化
- **Null Space**: 冗長性がある場合の副次的動作空間

In [ ]:
# Jacobian Analysis and Manipulability
class JacobianAnalyzer:
    def __init__(self, L1=1.0, L2=1.0):
        self.L1 = L1
        self.L2 = L2
        
    def forward_kinematics(self, theta1, theta2):
        """Compute joint and end-effector positions"""
        x1 = self.L1 * np.cos(theta1)
        y1 = self.L1 * np.sin(theta1)
        x2 = x1 + self.L2 * np.cos(theta1 + theta2)
        y2 = y1 + self.L2 * np.sin(theta1 + theta2)
        return x1, y1, x2, y2
    
    def compute_jacobian(self, theta1, theta2):
        """Compute the 2x2 Jacobian matrix"""
        J = np.array([
            [-self.L1*np.sin(theta1) - self.L2*np.sin(theta1+theta2), 
             -self.L2*np.sin(theta1+theta2)],
            [self.L1*np.cos(theta1) + self.L2*np.cos(theta1+theta2),  
             self.L2*np.cos(theta1+theta2)]
        ])
        return J
    
    def compute_manipulability_metrics(self, theta1, theta2):
        """Compute manipulability ellipsoid parameters"""
        J = self.compute_jacobian(theta1, theta2)
        
        # Compute J @ J^T for velocity ellipsoid
        A = J @ J.T
        
        # Eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eig(A)
        
        # Sort by eigenvalue magnitude
        idx = eigenvalues.argsort()[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # Ellipse parameters
        width = 2 * np.sqrt(eigenvalues[0]) if eigenvalues[0] > 0 else 0
        height = 2 * np.sqrt(eigenvalues[1]) if eigenvalues[1] > 0 else 0
        angle = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
        
        # Manipulability measure (determinant)
        manipulability = np.sqrt(np.linalg.det(A)) if np.linalg.det(A) > 0 else 0
        
        # Condition number
        if eigenvalues[1] > 1e-10:
            condition = np.sqrt(eigenvalues[0] / eigenvalues[1])
        else:
            condition = np.inf
            
        return width, height, angle, manipulability, condition, eigenvalues, eigenvectors

def demonstrate_jacobian_analysis():
    """Comprehensive Jacobian and manipulability demonstration"""
    analyzer = JacobianAnalyzer()
    
    fig = plt.figure(figsize=(16, 12))
    
    # Main robot visualization with manipulability ellipsoid
    ax1 = plt.subplot(221)
    ax1.set_title('Robot & Manipulability Ellipsoid', fontsize=14, fontweight='bold')
    ax1.set_xlabel('x (m)')
    ax1.set_ylabel('y (m)')
    ax1.set_xlim([-2.5, 2.5])
    ax1.set_ylim([-2.5, 2.5])
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    
    # Jacobian columns in velocity space
    ax2 = plt.subplot(222)
    ax2.set_title('Jacobian Columns (Joint Vel → EE Twist)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('vx (m/s)')
    ax2.set_ylabel('vy (m/s)')
    ax2.set_xlim([-3, 3])
    ax2.set_ylim([-3, 3])
    ax2.set_aspect('equal')
    ax2.grid(True, alpha=0.3)
    
    # Manipulability across workspace
    ax3 = plt.subplot(223)
    ax3.set_title('Manipulability Across Workspace', fontsize=14, fontweight='bold')
    ax3.set_xlabel('θ₁ (rad)')
    ax3.set_ylabel('θ₂ (rad)')
    ax3.set_xlim([0, 2*np.pi])
    ax3.set_ylim([0, 2*np.pi])
    
    # Condition number across workspace  
    ax4 = plt.subplot(224)
    ax4.set_title('Condition Number (Singularity Analysis)', fontsize=14, fontweight='bold')
    ax4.set_xlabel('θ₁ (rad)')
    ax4.set_ylabel('θ₂ (rad)')
    ax4.set_xlim([0, 2*np.pi])
    ax4.set_ylim([0, 2*np.pi])
    
    # Create workspace analysis
    theta1_range = np.linspace(0, 2*np.pi, 50)
    theta2_range = np.linspace(0, 2*np.pi, 50)
    T1, T2 = np.meshgrid(theta1_range, theta2_range)
    
    manipulability_map = np.zeros_like(T1)
    condition_map = np.zeros_like(T1)
    
    for i in range(len(theta1_range)):
        for j in range(len(theta2_range)):
            _, _, _, manip, cond, _, _ = analyzer.compute_manipulability_metrics(T1[j,i], T2[j,i])
            manipulability_map[j,i] = manip
            condition_map[j,i] = min(cond, 20)  # Cap for visualization
    
    # Plot manipulability map
    im1 = ax3.contourf(T1, T2, manipulability_map, levels=20, cmap='viridis')
    plt.colorbar(im1, ax=ax3, label='Manipulability')
    
    # Plot condition number map
    im2 = ax4.contourf(T1, T2, condition_map, levels=20, cmap='plasma')
    plt.colorbar(im2, ax=ax4, label='Condition Number')
    
    # Mark singular configurations
    # θ₂ = 0 (fully extended)
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='θ₂=0 (extended)')
    # θ₂ = π (fully folded)  
    ax4.axhline(y=np.pi, color='red', linestyle='--', alpha=0.7, label='θ₂=π (folded)')
    ax4.legend()
    
    # Show specific configuration
    theta1_demo = np.pi/4
    theta2_demo = np.pi/3
    
    # Robot configuration
    x1, y1, x2, y2 = analyzer.forward_kinematics(theta1_demo, theta2_demo)
    ax1.plot([0, x1], [0, y1], 'k-', linewidth=4, label='Link 1')
    ax1.plot([x1, x2], [y1, y2], 'k-', linewidth=4, label='Link 2')
    ax1.plot(0, 0, 'ko', markersize=10)
    ax1.plot(x1, y1, 'ko', markersize=8)
    ax1.plot(x2, y2, 'ro', markersize=10, label='End-effector')
    
    # Manipulability ellipsoid
    width, height, angle, manip, cond, eigenvals, eigenvecs = analyzer.compute_manipulability_metrics(theta1_demo, theta2_demo)
    
    if manip > 0.01:
        ellipse = Ellipse((x2, y2), width, height, 
                         angle=np.degrees(angle), 
                         fill=False, edgecolor='blue', linewidth=2)
        ax1.add_patch(ellipse)
        
        # Show principal directions
        for i, (eigenval, eigenvec) in enumerate(zip(eigenvals, eigenvecs.T)):
            scale = np.sqrt(eigenval) * 0.5
            ax1.arrow(x2, y2, scale * eigenvec[0], scale * eigenvec[1],
                     head_width=0.05, head_length=0.05,
                     fc=f'C{i}', ec=f'C{i}', alpha=0.7)
    
    # Jacobian columns
    J = analyzer.compute_jacobian(theta1_demo, theta2_demo)
    col1, col2 = J[:, 0], J[:, 1]
    
    ax2.arrow(0, 0, col1[0], col1[1], head_width=0.1, head_length=0.1, 
             fc='red', ec='red', alpha=0.7, linewidth=2, label='J₁ (θ̇₁=1)')
    ax2.arrow(0, 0, col2[0], col2[1], head_width=0.1, head_length=0.1, 
             fc='blue', ec='blue', alpha=0.7, linewidth=2, label='J₂ (θ̇₂=1)')
    
    # Unit circle constraint
    circle = plt.Circle((0, 0), 1, fill=False, edgecolor='gray', linestyle='--', alpha=0.5)
    ax2.add_patch(circle)
    ax2.text(0.7, 0.7, '‖θ̇‖=1', fontsize=9, color='gray')
    
    # Velocity ellipsoid in velocity space
    if manip > 0.01:
        ellipse_vel = Ellipse((0, 0), width, height, 
                             angle=np.degrees(angle), 
                             fill=True, facecolor='cyan', alpha=0.3,
                             edgecolor='blue', linewidth=1)
        ax2.add_patch(ellipse_vel)
    
    ax1.legend()
    ax2.legend()
    
    # Add configuration info
    ax1.text(0.02, 0.98, f'Configuration: θ₁={theta1_demo:.2f}, θ₂={theta2_demo:.2f}\\n'\
                         f'Manipulability: {manip:.3f}\\n'\
                         f'Condition: {cond:.2f}',
            transform=ax1.transAxes, fontsize=10,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return fig, analyzer

# Run the demonstration
print("📐 Jacobian Analysis and Manipulability")
print("=" * 40)
fig, analyzer = demonstrate_jacobian_analysis()

print("\\n🔍 Analysis Results:")
theta1_test, theta2_test = np.pi/4, np.pi/3
J = analyzer.compute_jacobian(theta1_test, theta2_test)
width, height, angle, manip, cond, eigenvals, eigenvecs = analyzer.compute_manipulability_metrics(theta1_test, theta2_test)

print(f"Jacobian Matrix:")
print(J)
print(f"\\nManipulability Measure: {manip:.4f}")
print(f"Condition Number: {cond:.2f}")
print(f"Eigenvalues: {eigenvals}")

print("\\n💡 Key Insights:")
print("• Large ellipsoid = high manipulability = good dexterity")
print("• High condition number = near singularity = poor performance")
print("• Ellipsoid orientation shows preferred motion directions")
print("• Singular configurations: θ₂=0 (extended), θ₂=π (folded)")

## 📈 7. Trajectory Generation and Time Scaling

### Course 4: 軌道生成 - パスと時間スケーリングの分離

**基本概念**: **軌道 = パス（幾何学）+ 時間スケーリング**

- **パス**: 空間内の幾何学的経路（時間に依存しない）
- **時間スケーリング**: s(t): [0,T] → [0,1] （単調増加、C²連続）

**3つの時間プロファイル**:
1. **Quintic**: 滑らかだが遅い
2. **Trapezoidal**: 速度制限下で時間最適
3. **S-curve**: ジャーク制限で快適性重視

In [ ]:
# Trajectory Generation - Path and Time Scaling
def quintic_time_scaling(T, t):
    """Quintic polynomial time scaling - smooth but slow"""
    tau = t / T
    s = 10 * tau**3 - 15 * tau**4 + 6 * tau**5
    s_dot = (30 * tau**2 - 60 * tau**3 + 30 * tau**4) / T
    s_ddot = (60 * tau - 180 * tau**2 + 120 * tau**3) / (T**2)
    return s, s_dot, s_ddot

def trapezoidal_time_scaling(T, t, acc_time_ratio=0.3):
    """Trapezoidal velocity profile - time optimal for velocity limits"""
    t_acc = T * acc_time_ratio
    t_const = T - 2 * t_acc
    v_max = 1.0 / (T - t_acc)
    
    if t <= t_acc:
        # Acceleration phase
        a = v_max / t_acc
        s = 0.5 * a * t**2
        s_dot = a * t
        s_ddot = a
    elif t <= T - t_acc:
        # Constant velocity phase
        s = 0.5 * v_max * t_acc + v_max * (t - t_acc)
        s_dot = v_max
        s_ddot = 0
    else:
        # Deceleration phase
        t_dec = t - (T - t_acc)
        a = v_max / t_acc
        s = 0.5 * v_max * t_acc + v_max * t_const + v_max * t_dec - 0.5 * a * t_dec**2
        s_dot = v_max - a * t_dec
        s_ddot = -a
    
    return s, s_dot, s_ddot

def s_curve_time_scaling(T, t, jerk_time_ratio=0.2):
    """S-curve (7-segment) profile - jerk limited for comfort"""
    tj = T * jerk_time_ratio
    
    if t <= tj:
        # Jerk phase 1
        j = 2.0 / (tj * T)
        s = (j * t**3) / 6
        s_dot = 0.5 * j * t**2
        s_ddot = j * t
    elif t <= T/2:
        # Constant acceleration
        t1 = t - tj
        a_max = 2.0 * tj / (tj * T)
        v0 = 0.5 * a_max * tj
        s0 = a_max * tj**3 / 6
        s = s0 + v0 * t1 + 0.5 * a_max * t1**2
        s_dot = v0 + a_max * t1
        s_ddot = a_max
    elif t <= T - tj:
        # Use symmetry for second half
        s, s_dot, s_ddot = s_curve_time_scaling(T, T - t, jerk_time_ratio)
        s = 1 - s
        s_dot = -s_dot
        s_ddot = -s_ddot
    else:
        # Final jerk phase
        t_end = T - t
        j = 2.0 / (tj * T)
        s = 1 - (j * t_end**3) / 6
        s_dot = 0.5 * j * t_end**2
        s_ddot = -j * t_end
    
    return s, s_dot, s_ddot

class PathTrajectoryDemo:
    def __init__(self):
        # Define figure-8 path in 2D
        self.path_parameter = np.linspace(0, 1, 200)
        self.path_x = 2 * np.sin(2 * np.pi * self.path_parameter)
        self.path_y = np.sin(4 * np.pi * self.path_parameter)
        
        # Time parameters
        self.T = 5.0  # Total time
        self.dt = 0.05
        self.time_array = np.arange(0, self.T, self.dt)
        
    def get_path_point(self, s):
        """Get (x, y) position on path at parameter s"""
        s = np.clip(s, 0, 1)  # Ensure s is in [0,1]
        
        if s >= 1:
            return self.path_x[-1], self.path_y[-1]
        
        # Linear interpolation
        idx = int(s * (len(self.path_parameter) - 1))
        if idx >= len(self.path_parameter) - 1:
            return self.path_x[-1], self.path_y[-1]
        
        alpha = s * (len(self.path_parameter) - 1) - idx
        x = (1 - alpha) * self.path_x[idx] + alpha * self.path_x[idx + 1]
        y = (1 - alpha) * self.path_y[idx] + alpha * self.path_y[idx + 1]
        return x, y

def demonstrate_trajectory_generation():
    """Comprehensive trajectory generation demonstration"""
    demo = PathTrajectoryDemo()
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Path visualization
    ax1.set_title('Geometric Path (Independent of Time)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_xlim([-3, 3])
    ax1.set_ylim([-1.5, 1.5])
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    
    # Time scaling profiles
    ax2.set_title('Time Scaling Profiles s(t)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Path Parameter s')
    ax2.set_xlim([0, demo.T])
    ax2.set_ylim([0, 1.1])
    ax2.grid(True, alpha=0.3)
    
    # Velocity profiles
    ax3.set_title('Velocity Profiles ṡ(t)', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Velocity ṡ')
    ax3.set_xlim([0, demo.T])
    ax3.set_ylim([0, 0.6])
    ax3.grid(True, alpha=0.3)
    
    # Acceleration profiles
    ax4.set_title('Acceleration Profiles s̈(t)', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Acceleration s̈')
    ax4.set_xlim([0, demo.T])
    ax4.set_ylim([-0.5, 0.5])
    ax4.grid(True, alpha=0.3)
    
    # Draw static path
    ax1.plot(demo.path_x, demo.path_y, 'k-', linewidth=2, alpha=0.5, label='Figure-8 Path')
    ax1.plot(demo.path_x[0], demo.path_y[0], 'go', markersize=10, label='Start')
    ax1.plot(demo.path_x[-1], demo.path_y[-1], 'ro', markersize=10, label='End')
    
    # Pre-compute all time scaling profiles
    profiles = {
        'Quintic': {'color': 'blue', 'data': [], 'style': '-'},
        'Trapezoidal': {'color': 'red', 'data': [], 'style': '--'},
        'S-curve': {'color': 'green', 'data': [], 'style': ':'}
    }
    
    for t in demo.time_array:
        s_q, v_q, a_q = quintic_time_scaling(demo.T, t)
        profiles['Quintic']['data'].append((s_q, v_q, a_q))
        
        s_t, v_t, a_t = trapezoidal_time_scaling(demo.T, t)
        profiles['Trapezoidal']['data'].append((s_t, v_t, a_t))
        
        s_s, v_s, a_s = s_curve_time_scaling(demo.T, t)
        profiles['S-curve']['data'].append((s_s, v_s, a_s))
    
    # Draw complete profiles
    for name, profile in profiles.items():
        s_vals = [d[0] for d in profile['data']]
        v_vals = [d[1] for d in profile['data']]
        a_vals = [d[2] for d in profile['data']]
        
        ax2.plot(demo.time_array, s_vals, color=profile['color'], 
                linestyle=profile['style'], label=name, linewidth=2)
        ax3.plot(demo.time_array, v_vals, color=profile['color'], 
                linestyle=profile['style'], label=name, linewidth=2)
        ax4.plot(demo.time_array, a_vals, color=profile['color'], 
                linestyle=profile['style'], label=name, linewidth=2)
    
    # Show sample trajectories on path
    sample_times = [1.0, 2.0, 3.0, 4.0]
    for i, t in enumerate(sample_times):
        if t < demo.T:
            t_idx = int(t / demo.dt)
            
            for name, profile in profiles.items():
                s = profile['data'][t_idx][0]
                x, y = demo.get_path_point(s)
                ax1.plot(x, y, 'o', color=profile['color'], markersize=6, alpha=0.7)
    
    # Add legends and annotations
    ax1.legend()
    ax2.legend()
    ax3.legend()
    ax4.legend()
    
    # Add zero lines for reference
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Performance comparison
    print("\\n⚡ Performance Comparison:")
    print("Profile         | Max Velocity | Max Acceleration | Smoothness")
    print("-" * 65)
    
    for name, profile in profiles.items():
        v_vals = [abs(d[1]) for d in profile['data']]
        a_vals = [abs(d[2]) for d in profile['data']]
        max_v = max(v_vals) if v_vals else 0
        max_a = max(a_vals) if a_vals else 0
        smoothness = "High" if name == "Quintic" else "Medium" if name == "S-curve" else "Low"
        print(f"{name:12} | {max_v:11.3f} | {max_a:15.3f} | {smoothness}")
    
    return fig

# Run trajectory generation demo
print("📈 Trajectory Generation - Path vs Time Scaling")
print("=" * 50)
fig = demonstrate_trajectory_generation()

print("\\n🎯 Key Design Insights:")
print("• Decoupling path from timing allows independent optimization")  
print("• Quintic: Smooth but slow - good for precision tasks")
print("• Trapezoidal: Time-optimal for velocity constraints - industrial standard")
print("• S-curve: Jerk-limited for comfort - used in elevators, CNC machines")
print("• Same path + different timing = completely different robot behavior!")

print("\\n🏭 Industrial Applications:")
print("• Pick-and-place: Trapezoidal for speed")
print("• Welding: Quintic for smoothness") 
print("• Human-robot interaction: S-curve for comfort")

## 🚗 8. Mobile Robot Kinematics - Mecanum Wheels

### Course 6: Mecanum Wheel Robot - True Omnidirectional Motion

**Revolutionary Capability**: Independent control of vₓ, vᵧ, and ω enables holonomic motion

**Mecanum Kinematics**:
- **4 wheels** at 45° roller angles
- **H matrix** converts body twist to wheel velocities
- **Omnidirectional**: Move in any direction while rotating independently

**Motion Modes**: Forward, Sideways, Diagonal, Pure Rotation

In [ ]:
# Mecanum Wheel Robot Simulation
class MecanumRobot:
    """Four-wheel mecanum robot for omnidirectional motion"""
    
    def __init__(self, length=0.235, width=0.15):
        self.L = length  # Half-length
        self.W = width   # Half-width  
        self.wheel_radius = 0.05
        
        # Robot pose: [x, y, phi]
        self.pose = np.array([0.0, 0.0, 0.0])
        
        # H matrix for mecanum kinematics (body to wheel velocities)
        # Wheel configuration: 1(FL)---2(FR)
        #                      |         |
        #                      3(RL)---4(RR)
        self.H_inv = np.array([
            [-1/(self.L+self.W),  1,  -1],  # Wheel 1 (front-left)
            [ 1/(self.L+self.W),  1,   1],  # Wheel 2 (front-right)
            [ 1/(self.L+self.W),  1,  -1],  # Wheel 3 (rear-left)
            [-1/(self.L+self.W),  1,   1],  # Wheel 4 (rear-right)
        ]) / self.wheel_radius
        
        # Forward kinematics matrix (wheel velocities to body twist)
        self.H = np.linalg.pinv(self.H_inv)
        
    def body_to_wheel_velocities(self, vx, vy, omega):
        """Convert body twist to wheel velocities"""
        body_twist = np.array([omega, vx, vy])
        wheel_velocities = self.H_inv @ body_twist
        return wheel_velocities
    
    def wheel_to_body_velocities(self, wheel_velocities):
        """Convert wheel velocities to body twist"""
        body_twist = self.H @ wheel_velocities
        return body_twist[1], body_twist[2], body_twist[0]  # vx, vy, omega
    
    def update_pose(self, vx, vy, omega, dt):
        """Update robot pose given body velocities"""
        # Convert body velocities to world frame
        phi = self.pose[2]
        R = np.array([
            [np.cos(phi), -np.sin(phi)],
            [np.sin(phi),  np.cos(phi)]
        ])
        
        v_world = R @ np.array([vx, vy])
        
        # Update pose
        self.pose[0] += v_world[0] * dt
        self.pose[1] += v_world[1] * dt
        self.pose[2] += omega * dt
        
        # Wrap angle
        self.pose[2] = np.arctan2(np.sin(self.pose[2]), np.cos(self.pose[2]))
        
    def get_wheel_positions(self):
        """Get wheel positions in world frame"""
        x, y, phi = self.pose
        R = np.array([
            [np.cos(phi), -np.sin(phi)],
            [np.sin(phi),  np.cos(phi)]
        ])
        
        # Wheel positions in body frame
        wheel_body = np.array([
            [-self.L,  self.W],  # Front-left
            [ self.L,  self.W],  # Front-right
            [-self.L, -self.W],  # Rear-left
            [ self.L, -self.W],  # Rear-right
        ])
        
        # Transform to world frame
        wheel_world = []
        for wb in wheel_body:
            ww = R @ wb + np.array([x, y])
            wheel_world.append(ww)
            
        return wheel_world

def draw_mecanum_robot(ax, robot, wheel_velocities=None, scale=1.0):
    """Draw mecanum robot at current pose"""
    x, y, phi = robot.pose
    
    # Robot body dimensions
    robot_width = 0.3 * scale
    robot_length = 0.47 * scale
    
    # Create rotation matrix
    R = np.array([
        [np.cos(phi), -np.sin(phi)],
        [np.sin(phi),  np.cos(phi)]
    ])
    
    # Body corners in body frame
    corners_body = np.array([
        [-robot_length/2, -robot_width/2],
        [ robot_length/2, -robot_width/2],
        [ robot_length/2,  robot_width/2],
        [-robot_length/2,  robot_width/2],
        [-robot_length/2, -robot_width/2]
    ])
    
    # Transform to world frame
    corners_world = np.array([R @ c + np.array([x, y]) for c in corners_body])
    
    # Draw body
    body = patches.Polygon(corners_world, closed=True, 
                          edgecolor='black', facecolor='lightblue', 
                          linewidth=2, alpha=0.7)
    ax.add_patch(body)
    
    # Draw wheels
    wheel_positions = robot.get_wheel_positions()
    wheel_colors = ['red', 'green', 'blue', 'orange']
    wheel_labels = ['FL', 'FR', 'RL', 'RR']
    
    for i, (wp, color, label) in enumerate(zip(wheel_positions, wheel_colors, wheel_labels)):
        # Draw wheel
        wheel = Circle(wp, 0.03*scale, color=color, alpha=0.8)
        ax.add_patch(wheel)
        
        # Draw roller direction (45 degree lines)
        roller_angle = phi + ((-1)**(i//2)) * np.pi/4
        dx = 0.04 * scale * np.cos(roller_angle)
        dy = 0.04 * scale * np.sin(roller_angle)
        ax.plot([wp[0]-dx, wp[0]+dx], [wp[1]-dy, wp[1]+dy], 
               color=color, linewidth=2, alpha=0.7)
        
        # Label wheels
        ax.text(wp[0], wp[1]-0.08*scale, label, ha='center', fontsize=8, fontweight='bold')
        
        # Show wheel velocity if provided
        if wheel_velocities is not None and abs(wheel_velocities[i]) > 0.1:
            arrow_scale = np.clip(wheel_velocities[i] / 10, -1, 1) * 0.05 * scale
            arrow_angle = phi + np.pi/2 if wheel_velocities[i] > 0 else phi - np.pi/2
            ax.arrow(wp[0], wp[1], 
                   arrow_scale * np.cos(arrow_angle),
                   arrow_scale * np.sin(arrow_angle),
                   head_width=0.015*scale, head_length=0.015*scale,
                   fc=color, ec=color, alpha=0.9)
    
    # Draw heading arrow
    ax.arrow(x, y, 0.15*scale*np.cos(phi), 0.15*scale*np.sin(phi),
            head_width=0.03*scale, head_length=0.03*scale,
            fc='black', ec='black', alpha=0.8, linewidth=2)

def demonstrate_mecanum_motion():
    """Demonstrate different mecanum motion modes"""
    robot = MecanumRobot()
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Motion modes to demonstrate
    motion_modes = {
        'Forward': (0.5, 0, 0),
        'Sideways': (0, 0.5, 0), 
        'Diagonal': (0.35, 0.35, 0),
        'Rotation': (0, 0, 1.0)
    }
    
    axes = [ax1, ax2, ax3, ax4]
    mode_names = list(motion_modes.keys())
    
    for ax, mode_name in zip(axes, mode_names):
        ax.set_title(f'{mode_name} Motion', fontsize=14, fontweight='bold')
        ax.set_xlabel('x (m)')
        ax.set_ylabel('y (m)')
        ax.set_xlim([-1.5, 1.5])
        ax.set_ylim([-1.5, 1.5])
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        
        # Reset robot pose
        robot.pose = np.array([0.0, 0.0, 0.0])
        
        # Get motion command
        vx_cmd, vy_cmd, omega_cmd = motion_modes[mode_name]
        wheel_velocities = robot.body_to_wheel_velocities(vx_cmd, vy_cmd, omega_cmd)
        
        # Simulate trajectory
        dt = 0.1
        trajectory = []
        for t in range(30):  # 3 seconds
            trajectory.append(robot.pose[:2].copy())
            robot.update_pose(vx_cmd, vy_cmd, omega_cmd, dt)
        
        # Draw trajectory
        if len(trajectory) > 1:
            traj = np.array(trajectory)
            ax.plot(traj[:, 0], traj[:, 1], 'b-', linewidth=2, alpha=0.6, label='Trajectory')
            
        # Draw robot at final position
        draw_mecanum_robot(ax, robot, wheel_velocities)
        
        # Show velocity command
        ax.text(0.02, 0.98, f'Command:\\nvx = {vx_cmd:.2f} m/s\\nvy = {vy_cmd:.2f} m/s\\nω = {omega_cmd:.2f} rad/s',
                transform=ax.transAxes, fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
        
        # Show wheel velocities
        wheel_text = f'Wheels: [{wheel_velocities[0]:.1f}, {wheel_velocities[1]:.1f}, {wheel_velocities[2]:.1f}, {wheel_velocities[3]:.1f}] rad/s'
        ax.text(0.02, 0.02, wheel_text, transform=ax.transAxes, fontsize=9,
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
        
        # Draw coordinate system
        ax.arrow(-1.3, -1.3, 0.3, 0, head_width=0.05, head_length=0.05, fc='red', ec='red')
        ax.arrow(-1.3, -1.3, 0, 0.3, head_width=0.05, head_length=0.05, fc='green', ec='green')
        ax.text(-1.3, -1.5, 'x', fontsize=10, color='red', ha='center')
        ax.text(-1.5, -1.3, 'y', fontsize=10, color='green', ha='center')
        
        ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    return fig

def analyze_mecanum_kinematics():
    """Analyze mecanum kinematics matrices"""
    robot = MecanumRobot()
    
    print("🔍 Mecanum Kinematics Analysis:")
    print("=" * 40)
    print("\\nH_inv matrix (body twist → wheel velocities):")
    print(robot.H_inv)
    print("\\nH matrix (wheel velocities → body twist):")  
    print(robot.H)
    
    # Test different commands
    test_commands = [
        ("Forward", 1, 0, 0),
        ("Sideways", 0, 1, 0),
        ("Rotation", 0, 0, 1),
        ("Diagonal", 0.707, 0.707, 0)
    ]
    
    print("\\n📊 Velocity Mapping Examples:")
    print("Mode       | Body Twist [ω,vx,vy] | Wheel Velocities [FL,FR,RL,RR]")
    print("-" * 70)
    
    for name, vx, vy, omega in test_commands:
        wheel_vels = robot.body_to_wheel_velocities(vx, vy, omega)
        print(f"{name:10} | [{omega:4.1f},{vx:4.1f},{vy:4.1f}]        | [{wheel_vels[0]:5.1f},{wheel_vels[1]:5.1f},{wheel_vels[2]:5.1f},{wheel_vels[3]:5.1f}]")
    
    return robot

# Run mecanum demonstration
print("🚗 Mecanum Wheel Robot - Omnidirectional Motion")
print("=" * 50)
fig = demonstrate_mecanum_motion()
robot = analyze_mecanum_kinematics()

print("\\n💡 Key Insights:")
print("• Mecanum wheels enable holonomic motion (translation + rotation)")
print("• 45° rollers create complex wheel-ground interaction")  
print("• Independent control of vx, vy, ω gives 3 DOF mobility")
print("• Wheel velocity patterns are unique for each motion mode")
print("• Real-world applications: warehouses, hospitals, tight spaces")

## 🏆 9. Capstone Project Integration - KUKA youBot

### Course 6: Capstone Project - Pick and Place Implementation

**統合プロジェクト**: 5つのマイルストーンで6コースの知識を結集

1. **NextState**: オドメトリー更新
2. **TrajectoryGeneration**: 8セグメント軌道生成  
3. **FeedforwardControl**: フィードフォワード制御
4. **FeedbackControl**: PI制御の追加
5. **FullSystem**: 統合システムの実行

**KUKA youBot**: 4輪メカナムベース + 5自由度アーム = 9 DOF

In [ ]:
# Capstone Project - KUKA youBot Pick and Place
# This section demonstrates the final integration project

def demonstrate_capstone_overview():
    """Visual overview of the capstone project structure"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Project Timeline
    ax1.set_title('Capstone Project Timeline', fontsize=14, fontweight='bold')
    ax1.set_xlim([0, 10])
    ax1.set_ylim([0, 6])
    
    milestones = [
        (1, 5, "Milestone 1\\nNextState", "Update robot configuration\\nfrom wheel speeds"),
        (3, 4, "Milestone 2\\nTrajectory Gen", "8-segment reference\\ntrajectory generation"),
        (5, 3, "Milestone 3\\nFeedforward", "Open-loop control\\nusing desired trajectory"),
        (7, 2, "Milestone 4\\nFeedback", "Add PI control for\\nerror correction"),
        (9, 1, "Integration", "Complete pick-and-place\\nsystem validation")
    ]
    
    for i, (x, y, title, desc) in enumerate(milestones):
        # Draw milestone box
        rect = Rectangle((x-0.4, y-0.3), 0.8, 0.6, 
                        facecolor=f'C{i}', alpha=0.7, edgecolor='black')
        ax1.add_patch(rect)
        ax1.text(x, y, title, ha='center', va='center', fontsize=10, fontweight='bold')
        ax1.text(x, y-0.8, desc, ha='center', va='top', fontsize=8)
        
        # Draw arrows between milestones
        if i < len(milestones) - 1:
            ax1.arrow(x+0.4, y, 1.2, 0, head_width=0.1, head_length=0.2, 
                     fc='gray', ec='gray', alpha=0.7)
    
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.axis('off')
    
    # System Architecture
    ax2.set_title('KUKA youBot System Architecture', fontsize=14, fontweight='bold')
    ax2.set_xlim([0, 10])
    ax2.set_ylim([0, 10])
    
    # Draw youBot schematic
    # Mobile base
    base_rect = Rectangle((2, 1), 6, 2, facecolor='lightblue', edgecolor='black', linewidth=2)
    ax2.add_patch(base_rect)
    ax2.text(5, 2, 'Mecanum Base\\n4 DOF (x,y,φ)', ha='center', va='center', fontsize=10)
    
    # Wheels
    wheel_positions = [(2.5, 0.5), (7.5, 0.5), (2.5, 3.5), (7.5, 3.5)]
    for wx, wy in wheel_positions:
        wheel = Circle((wx, wy), 0.2, facecolor='gray', edgecolor='black')
        ax2.add_patch(wheel)
    
    # Arm
    arm_segments = [(5, 3), (5, 5), (6, 7), (7.5, 8)]
    for i in range(len(arm_segments)-1):
        ax2.plot([arm_segments[i][0], arm_segments[i+1][0]], 
                [arm_segments[i][1], arm_segments[i+1][1]], 'k-', linewidth=4)
        # Joint circles
        joint = Circle(arm_segments[i+1], 0.15, facecolor='red', edgecolor='black')
        ax2.add_patch(joint)
    
    ax2.text(8, 8.5, '5-DOF Arm', ha='center', fontsize=10, fontweight='bold')
    ax2.text(5, 0.2, 'Total: 9 DOF System', ha='center', fontsize=12, fontweight='bold', 
             bbox=dict(boxstyle='round', facecolor='yellow'))
    
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.axis('off')
    
    # Control Architecture
    ax3.set_title('Control System Architecture', fontsize=14, fontweight='bold')
    ax3.set_xlim([0, 10])
    ax3.set_ylim([0, 8])
    
    # Control blocks
    blocks = [
        (2, 6.5, "Reference\\nTrajectory", 'lightgreen'),
        (5, 6.5, "Feedforward\\nController", 'lightblue'),  
        (8, 6.5, "Robot\\nSystem", 'lightcoral'),
        (5, 4, "Feedback\\nController", 'lightyellow'),
        (2, 1.5, "Actual\\nConfiguration", 'lightgray')
    ]
    
    for x, y, text, color in blocks:
        rect = Rectangle((x-0.8, y-0.5), 1.6, 1, facecolor=color, edgecolor='black')
        ax3.add_patch(rect)
        ax3.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')
    
    # Control arrows
    arrows = [
        ((2.8, 6.5), (4.2, 6.5)),  # Ref → FF
        ((5.8, 6.5), (7.2, 6.5)),  # FF → Robot
        ((8, 6), (8, 2.5)),        # Robot output
        ((2.8, 2), (4.2, 4)),      # Error to FB
        ((5, 4.5), (5, 6))         # FB to FF
    ]
    
    for (x1, y1), (x2, y2) in arrows:
        ax3.arrow(x1, y1, x2-x1, y2-y1, head_width=0.15, head_length=0.2, 
                 fc='blue', ec='blue', alpha=0.7)
    
    ax3.text(1, 3.2, 'Error', ha='center', fontsize=9, color='blue')
    ax3.text(9, 4, 'Feedback', ha='center', fontsize=9, color='blue', rotation=-90)
    
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.axis('off')
    
    # Performance Metrics
    ax4.set_title('Key Performance Metrics', fontsize=14, fontweight='bold')
    
    # Sample performance data
    metrics = ['Position Error', 'Orientation Error', 'Trajectory Smoothness', 'Execution Time']
    feedforward_scores = [0.3, 0.4, 0.8, 0.9]
    feedback_scores = [0.9, 0.85, 0.75, 0.8]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    ax4.bar(x - width/2, feedforward_scores, width, label='Feedforward Only', alpha=0.7)
    ax4.bar(x + width/2, feedback_scores, width, label='Feedforward + Feedback', alpha=0.7)
    
    ax4.set_ylabel('Performance Score')
    ax4.set_xticks(x)
    ax4.set_xticklabels(metrics, rotation=45, ha='right')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    ax4.set_ylim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    return fig

def demonstrate_8_segment_trajectory():
    """Show the 8-segment pick-and-place trajectory"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Task space trajectory
    ax1.set_title('8-Segment Pick-and-Place Trajectory', fontsize=14, fontweight='bold')
    ax1.set_xlabel('x (m)')
    ax1.set_ylabel('y (m)')
    ax1.set_xlim([-1, 1])
    ax1.set_ylim([-0.5, 1.5])
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    
    # Define key waypoints
    standoff_height = 0.2
    grasp_height = 0.025
    
    waypoints = {
        'initial': np.array([0, -0.5, 0.5]),
        'cube_standoff': np.array([1, 0, standoff_height]),
        'cube_grasp': np.array([1, 0, grasp_height]),
        'cube_up': np.array([1, 0, standoff_height]),
        'goal_standoff': np.array([0, 1, standoff_height]),
        'goal_place': np.array([0, 1, grasp_height]),
        'goal_up': np.array([0, 1, standoff_height]),
        'final': np.array([-1, 0, 0.5])
    }
    
    # 8 segments
    segments = [
        ('initial', 'cube_standoff', '1. Move to cube standoff'),
        ('cube_standoff', 'cube_grasp', '2. Move down to grasp'),
        ('cube_grasp', 'cube_grasp', '3. Close gripper'),
        ('cube_grasp', 'cube_up', '4. Lift cube'),
        ('cube_up', 'goal_standoff', '5. Transport to goal'),
        ('goal_standoff', 'goal_place', '6. Lower to place'),
        ('goal_place', 'goal_place', '7. Open gripper'),
        ('goal_place', 'final', '8. Move to final position')
    ]
    
    # Draw trajectory segments
    colors = plt.cm.tab10(np.linspace(0, 1, 8))
    
    for i, (start_key, end_key, label) in enumerate(segments):
        start = waypoints[start_key]
        end = waypoints[end_key]
        
        if start_key == end_key:  # Gripper action
            ax1.plot(start[0], start[1], 'o', color=colors[i], markersize=12, 
                    label=label, alpha=0.8)
        else:  # Motion segment
            ax1.arrow(start[0], start[1], end[0]-start[0], end[1]-start[1],
                     head_width=0.03, head_length=0.03, fc=colors[i], ec=colors[i],
                     alpha=0.8, linewidth=2)
            ax1.plot([start[0], end[0]], [start[1], end[1]], 
                    color=colors[i], linewidth=3, alpha=0.7, label=label)
    
    # Mark key positions
    ax1.plot(1, 0, 's', color='red', markersize=15, alpha=0.7, label='Cube')
    ax1.plot(0, 1, 's', color='green', markersize=15, alpha=0.7, label='Goal')
    
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Joint space evolution (simplified 2D projection)
    ax2.set_title('Sample Joint Trajectories', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Joint Angle (rad)')
    ax2.grid(True, alpha=0.3)
    
    # Simulate joint trajectories (simplified)
    time_total = 30.0  # Total time
    segment_times = [3, 2, 1, 2, 8, 2, 1, 5]  # Time for each segment
    t = np.cumsum([0] + segment_times)
    
    # Sample joint profiles (5-DOF arm)
    joint_profiles = []
    for joint in range(5):
        profile = []
        for segment in range(8):
            if segment < 4:  # Approach and grasp
                angle = np.sin(joint * 0.5) * (segment + 1) * 0.3
            else:  # Transport and place  
                angle = np.cos(joint * 0.3) * (8 - segment) * 0.4
            profile.append(angle)
        joint_profiles.append(profile)
    
    # Plot joint trajectories
    for joint in range(5):
        ax2.plot(t[:-1], joint_profiles[joint], 'o-', linewidth=2, 
                label=f'Joint {joint+1}', alpha=0.8)
    
    # Mark segment transitions
    for i, segment_time in enumerate(t[:-1]):
        ax2.axvline(x=segment_time, color='gray', linestyle='--', alpha=0.5)
        if i < len(segments):
            ax2.text(segment_time + 0.5, ax2.get_ylim()[1] * 0.9, 
                    f'S{i+1}', fontsize=8, rotation=90)
    
    ax2.legend()
    ax2.set_xlim([0, time_total])
    
    plt.tight_layout()
    plt.show()
    
    return fig

# Run capstone demonstrations
print("🏆 KUKA youBot Capstone Project")
print("=" * 40)

fig1 = demonstrate_capstone_overview()
fig2 = demonstrate_8_segment_trajectory()

print("\\n📊 Project Statistics:")
print("• Total DOF: 9 (4 mobile base + 5 arm)")  
print("• Trajectory segments: 8")
print("• Control modes: Feedforward + Feedback")
print("• Coordinate frames: World, Body, End-effector")
print("• Libraries used: modern_robotics, numpy")

print("\\n🔄 Integration of All 6 Courses:")
print("1. Configuration Space: 9-DOF robot C-space")
print("2. Forward Kinematics: PoE for arm kinematics") 
print("3. Dynamics: Mass matrix for control")
print("4. Trajectory Generation: 8-segment quintic splines")
print("5. Grasping: Pick-and-place manipulation")
print("6. Mobile Robots: Mecanum wheel kinematics")

print("\\n🎯 Key Results:")
print("✅ Feedforward control handles most of the motion")
print("✅ Feedback provides error correction and disturbance rejection")  
print("✅ Smooth quintic trajectories ensure jerk-free motion")
print("✅ Integrated system successfully completes pick-and-place task")

print("\\n💡 Real-world Applications:")
print("• Warehouse automation")
print("• Manufacturing pick-and-place")  
print("• Service robotics")
print("• Research and education platforms")

## 🤖 10. Advanced Topics and Extensions

Beyond the core capstone project, modern robotics offers exciting frontiers for further exploration:

In [ ]:
def demonstrate_advanced_topics():
    """Showcase advanced robotics concepts and future directions"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Motion Planning Algorithms
    ax1.set_title('Motion Planning Algorithms', fontsize=14, fontweight='bold')
    ax1.set_xlim([0, 10])
    ax1.set_ylim([0, 10])
    
    # Sample-based planning visualization
    np.random.seed(42)
    
    # Obstacles
    obstacles = [
        Rectangle((2, 6), 3, 1, facecolor='red', alpha=0.7),
        Rectangle((6, 2), 1, 4, facecolor='red', alpha=0.7),
        Rectangle((1, 1), 1, 2, facecolor='red', alpha=0.7)
    ]
    
    for obs in obstacles:
        ax1.add_patch(obs)
    
    # Random samples for RRT
    samples_x = np.random.uniform(0, 10, 50)
    samples_y = np.random.uniform(0, 10, 50)
    ax1.scatter(samples_x, samples_y, c='lightblue', s=20, alpha=0.6, label='RRT Samples')
    
    # Planned path
    path_x = [0.5, 1.5, 3.5, 4.5, 7.5, 8.5, 9.5]
    path_y = [0.5, 3.5, 4.5, 5.5, 7.5, 8.5, 9.5]
    ax1.plot(path_x, path_y, 'g-', linewidth=3, label='Planned Path')
    ax1.plot(path_x[0], path_y[0], 'go', markersize=10, label='Start')
    ax1.plot(path_x[-1], path_y[-1], 'ro', markersize=10, label='Goal')
    
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Machine Learning in Robotics
    ax2.set_title('Machine Learning in Robotics', fontsize=14, fontweight='bold')
    
    # Neural network architecture visualization
    layers = [3, 5, 5, 2]  # Input, hidden1, hidden2, output
    layer_names = ['State\\n(q, q̇)', 'Hidden\\nLayer 1', 'Hidden\\nLayer 2', 'Action\\n(τ)']
    
    for i, (size, name) in enumerate(zip(layers, layer_names)):
        x = i * 2.5 + 1
        y_positions = np.linspace(2, 8, size)
        
        # Draw neurons
        for y in y_positions:
            circle = Circle((x, y), 0.2, facecolor='lightblue', edgecolor='blue')
            ax2.add_patch(circle)
        
        # Draw connections to next layer
        if i < len(layers) - 1:
            next_y_positions = np.linspace(2, 8, layers[i+1])
            for y1 in y_positions:
                for y2 in next_y_positions:
                    ax2.plot([x+0.2, x+2.3], [y1, y2], 'b-', alpha=0.3, linewidth=0.5)
        
        # Layer labels
        ax2.text(x, 1, name, ha='center', va='top', fontsize=10, fontweight='bold')
    
    ax2.text(5, 9, 'Deep Reinforcement Learning\\nfor Robot Control', 
             ha='center', fontsize=12, fontweight='bold',
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))
    
    ax2.set_xlim([0, 10])
    ax2.set_ylim([0, 10])
    ax2.axis('off')
    
    # Multi-Robot Systems
    ax3.set_title('Multi-Robot Coordination', fontsize=14, fontweight='bold')
    ax3.set_xlim([0, 10])
    ax3.set_ylim([0, 10])
    
    # Formation control visualization
    formation_center = np.array([5, 5])
    formation_positions = [
        formation_center + np.array([0, 1.5]),      # Leader
        formation_center + np.array([-1.3, -0.75]), # Follower 1
        formation_center + np.array([1.3, -0.75]),  # Follower 2
        formation_center + np.array([0, -1.5])      # Follower 3
    ]
    
    robot_colors = ['red', 'blue', 'green', 'orange']
    robot_labels = ['Leader', 'Robot 2', 'Robot 3', 'Robot 4']
    
    for i, (pos, color, label) in enumerate(zip(formation_positions, robot_colors, robot_labels)):
        # Robot body
        robot = Circle(pos, 0.3, facecolor=color, alpha=0.7, edgecolor='black')
        ax3.add_patch(robot)
        ax3.text(pos[0], pos[1], f'R{i+1}', ha='center', va='center', 
                fontsize=8, fontweight='bold', color='white')
        
        # Communication links
        if i > 0:  # Connect followers to leader
            ax3.plot([formation_positions[0][0], pos[0]], 
                    [formation_positions[0][1], pos[1]], 
                    'k--', alpha=0.5, linewidth=1)
    
    # Formation desired shape
    desired_shape = np.array(formation_positions + [formation_positions[0]])
    ax3.plot(desired_shape[:, 0], desired_shape[:, 1], 'k:', linewidth=2, alpha=0.7)
    
    ax3.text(5, 8.5, 'Distributed Formation Control', ha='center', fontsize=12, fontweight='bold')
    ax3.text(5, 1.5, 'Consensus Algorithms\\n• Graph Theory\\n• Distributed Control\\n• Fault Tolerance', 
             ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightcyan'))
    
    ax3.set_xlabel('x (m)')
    ax3.set_ylabel('y (m)')
    ax3.grid(True, alpha=0.3)
    
    # Soft Robotics and Bio-inspiration
    ax4.set_title('Soft Robotics & Bio-inspiration', fontsize=14, fontweight='bold')
    ax4.set_xlim([0, 10])
    ax4.set_ylim([0, 10])
    
    # Soft robot arm visualization
    t_vals = np.linspace(0, 2*np.pi, 100)
    
    # Continuum robot backbone
    backbone_x = 2 + 3 * np.sin(t_vals/4) * np.exp(-t_vals/8)
    backbone_y = 1 + t_vals * 0.8
    
    # Limit to display area
    valid_idx = backbone_y <= 9
    backbone_x = backbone_x[valid_idx]
    backbone_y = backbone_y[valid_idx]
    
    ax4.plot(backbone_x, backbone_y, 'purple', linewidth=8, alpha=0.7, label='Soft Robot Arm')
    
    # Actuator segments
    segment_positions = np.linspace(0, len(backbone_x)-1, 6, dtype=int)
    for i, idx in enumerate(segment_positions[:-1]):
        if idx < len(backbone_x):
            # Pressure chambers
            chamber1 = Circle((backbone_x[idx] - 0.2, backbone_y[idx]), 0.15, 
                            facecolor='red', alpha=0.6, edgecolor='darkred')
            chamber2 = Circle((backbone_x[idx] + 0.2, backbone_y[idx]), 0.15, 
                            facecolor='blue', alpha=0.6, edgecolor='darkblue')
            ax4.add_patch(chamber1)
            ax4.add_patch(chamber2)
    
    # Bio-inspired elements
    ax4.text(7, 7, '🐙 Octopus Arm\\nInspiration', fontsize=12, ha='center',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    ax4.text(7, 5, '🌱 Vine Growth\\nGrowing Robots', fontsize=12, ha='center',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    ax4.text(7, 3, '🐘 Elephant Trunk\\nContinuum Mechanics', fontsize=12, ha='center',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    ax4.text(2, 0.5, 'Key Technologies:\\n• Pneumatic Actuation\\n• Soft Materials\\n• Continuum Mechanics', 
             fontsize=10, bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    ax4.set_xlabel('x (m)')
    ax4.set_ylabel('y (m)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def demonstrate_future_directions():
    """Show cutting-edge research directions"""
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    
    ax.set_title('Future Directions in Modern Robotics', fontsize=16, fontweight='bold')
    ax.set_xlim([0, 12])
    ax.set_ylim([0, 10])
    
    # Research areas as interconnected nodes
    areas = [
        (2, 8, "Quantum\\nRobotics", 'quantum computing\\nfor optimization'),
        (6, 8.5, "Embodied AI", 'AI-robot\\nco-design'),
        (10, 7.5, "Swarm\\nIntelligence", 'emergent collective\\nbehaviors'),
        (1.5, 5.5, "Morphing\\nRobots", 'shape-changing\\nstructures'),
        (5, 6, "Digital\\nTwins", 'virtual-physical\\nsynchronization'),
        (9, 5, "Bio-hybrid\\nSystems", 'living-artificial\\nintegration'),
        (2.5, 2.5, "Molecular\\nRobots", 'nano-scale\\nmanipulation'),
        (6.5, 2, "Space\\nRobotics", 'extreme environment\\nadaptation'),
        (10, 2.5, "Surgical\\nMicrorobots", 'minimally invasive\\nmedicine')
    ]
    
    # Draw nodes
    for i, (x, y, title, desc) in enumerate(areas):
        # Main circle
        circle = Circle((x, y), 0.8, facecolor=f'C{i}', alpha=0.7, edgecolor='black', linewidth=2)
        ax.add_patch(circle)
        ax.text(x, y + 0.1, title, ha='center', va='center', fontsize=10, fontweight='bold')
        ax.text(x, y - 1.3, desc, ha='center', va='top', fontsize=8, style='italic')
    
    # Draw connections between related areas
    connections = [
        (0, 1),  # Quantum-AI
        (1, 2),  # AI-Swarm
        (1, 4),  # AI-Digital Twins
        (3, 4),  # Morphing-Digital Twins
        (4, 5),  # Digital Twins-Bio-hybrid
        (5, 8),  # Bio-hybrid-Surgical
        (6, 7),  # Molecular-Space
        (7, 8),  # Space-Surgical
        (0, 3),  # Quantum-Morphing
        (2, 5)   # Swarm-Bio-hybrid
    ]
    
    for i, j in connections:
        x1, y1 = areas[i][0], areas[i][1]
        x2, y2 = areas[j][0], areas[j][1]
        ax.plot([x1, x2], [y1, y2], 'gray', alpha=0.4, linewidth=1, linestyle='--')
    
    # Central theme
    ax.text(6, 5, '🚀 FUTURE\\nROBOTICS', ha='center', va='center', 
           fontsize=20, fontweight='bold', 
           bbox=dict(boxstyle='round', facecolor='gold', alpha=0.8))
    
    # Add timeline at bottom
    timeline_y = 0.5
    years = [2025, 2030, 2035, 2040, 2050]
    milestones = [
        "Advanced AI\\nIntegration",
        "Biological\\nHybrids",
        "Quantum-Enhanced\\nControl", 
        "Molecular\\nAssembly",
        "Conscious\\nMachines?"
    ]
    
    for year, milestone in zip(years, milestones):
        x_pos = (year - 2025) / 25 * 10 + 1
        ax.plot([x_pos, x_pos], [timeline_y - 0.2, timeline_y + 0.2], 'k-', linewidth=2)
        ax.text(x_pos, timeline_y - 0.4, str(year), ha='center', fontsize=10, fontweight='bold')
        ax.text(x_pos, timeline_y + 0.4, milestone, ha='center', va='bottom', fontsize=8)
    
    ax.plot([1, 11], [timeline_y, timeline_y], 'k-', linewidth=1)
    ax.text(6, 0.1, 'Timeline of Expected Breakthroughs', ha='center', fontsize=12, 
           fontweight='bold', style='italic')
    
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    return fig

# Demonstrate advanced topics
print("🚀 Advanced Topics in Modern Robotics")
print("=" * 40)

fig1 = demonstrate_advanced_topics()
fig2 = demonstrate_future_directions()

print("\\n🔬 Research Frontiers:")
print("• Motion Planning: RRT*, PRM, Neural Motion Planning")
print("• Machine Learning: Deep RL, Imitation Learning, Meta-Learning")
print("• Multi-Robot Systems: Swarm Intelligence, Distributed Control")
print("• Soft Robotics: Continuum Mechanics, Bio-inspiration")
print("• Human-Robot Interaction: Social Robotics, Collaborative Manipulation")

print("\\n🌟 Emerging Technologies:")
print("• Quantum Computing for Optimization")
print("• Bio-hybrid Systems")
print("• Morphing and Self-reconfiguring Robots")
print("• Digital Twins for Robotics")
print("• Molecular-scale Robotics")

print("\\n💡 Key Takeaways:")
print("✅ Modern robotics builds on solid mathematical foundations")
print("✅ Integration of multiple disciplines drives innovation")
print("✅ Future systems will be more adaptive and intelligent") 
print("✅ Bio-inspiration continues to guide new developments")
print("✅ The field is rapidly evolving with new possibilities")

## 📚 11. Conclusion and Learning Path

This comprehensive notebook has taken you through the complete journey of modern robotics, from fundamental mathematical concepts to advanced applications and future directions.

In [ ]:
def create_learning_summary():
    """Create a comprehensive summary of the learning journey"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Knowledge Structure Map
    ax1.set_title('Modern Robotics Knowledge Structure', fontsize=14, fontweight='bold')
    ax1.set_xlim([0, 10])
    ax1.set_ylim([0, 10])
    
    # Core concepts as hierarchical structure
    levels = [
        [(5, 1.5, "Linear Algebra\\n& Calculus", 'lightblue')],  # Foundation
        [(2, 3, "SO(3)\\nRotations", 'lightgreen'), 
         (5, 3, "SE(3)\\nRigid Body", 'lightgreen'),
         (8, 3, "Lie Theory\\nScrew Axes", 'lightgreen')],  # Mathematical foundations
        [(1.5, 5, "Configuration\\nSpace", 'lightyellow'),
         (3.5, 5, "Forward\\nKinematics", 'lightyellow'),
         (5.5, 5, "Jacobians", 'lightyellow'),
         (7.5, 5, "Dynamics", 'lightyellow'),
         (9, 5, "Trajectory\\nGeneration", 'lightyellow')],  # Core robotics
        [(3, 7, "Control\\nSystems", 'lightcoral'),
         (7, 7, "Mobile\\nRobots", 'lightcoral')],  # Applications
        [(5, 8.5, "KUKA youBot\\nCapstone", 'gold')]  # Integration
    ]
    
    # Draw knowledge nodes
    for level in levels:
        for x, y, text, color in level:
            circle = Circle((x, y), 0.6, facecolor=color, alpha=0.7, edgecolor='black')
            ax1.add_patch(circle)
            ax1.text(x, y, text, ha='center', va='center', fontsize=8, fontweight='bold')
    
    # Draw connections showing dependencies
    connections = [
        # Foundation to math
        ((5, 2.1), (2, 2.4)), ((5, 2.1), (5, 2.4)), ((5, 2.1), (8, 2.4)),
        # Math to core robotics  
        ((2, 3.6), (1.5, 4.4)), ((5, 3.6), (3.5, 4.4)), ((5, 3.6), (5.5, 4.4)),
        ((8, 3.6), (7.5, 4.4)), ((5, 3.6), (9, 4.4)),
        # Core to applications
        ((3.5, 5.6), (3, 6.4)), ((7.5, 5.6), (7, 6.4)),
        # Applications to capstone
        ((3, 7.6), (5, 7.9)), ((7, 7.6), (5, 7.9))
    ]
    
    for (x1, y1), (x2, y2) in connections:
        ax1.arrow(x1, y1, x2-x1, y2-y1, head_width=0.1, head_length=0.1, 
                 fc='gray', ec='gray', alpha=0.6)
    
    ax1.axis('off')
    
    # Skills Development Timeline
    ax2.set_title('Skills Development Journey', fontsize=14, fontweight='bold')
    
    skills = ['Mathematical\\nFoundations', 'Geometric\\nReasoning', 'Algorithmic\\nThinking', 
              'System\\nIntegration', 'Problem\\nSolving']
    progress = [0.95, 0.9, 0.85, 0.8, 0.9]  # Skill mastery levels
    
    y_pos = np.arange(len(skills))
    bars = ax2.barh(y_pos, progress, color=['lightblue', 'lightgreen', 'lightyellow', 
                                           'lightcoral', 'gold'], alpha=0.8)
    
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(skills)
    ax2.set_xlabel('Mastery Level')
    ax2.set_xlim([0, 1])
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add percentage labels
    for i, (bar, pct) in enumerate(zip(bars, progress)):
        ax2.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
                f'{pct*100:.0f}%', va='center', fontweight='bold')
    
    # Tools and Libraries Mastery
    ax3.set_title('Tools & Libraries Experience', fontsize=14, fontweight='bold')
    
    tools = ['NumPy', 'Matplotlib', 'Modern\\nRobotics', 'Jupyter\\nNotebooks', 'Python\\nProgramming']
    proficiency = [0.9, 0.85, 0.95, 0.8, 0.85]
    
    # Radar chart
    angles = np.linspace(0, 2*np.pi, len(tools), endpoint=False).tolist()
    proficiency += proficiency[:1]  # Complete the circle
    angles += angles[:1]
    
    ax3.plot(angles, proficiency, 'o-', linewidth=2, color='blue', alpha=0.7)
    ax3.fill(angles, proficiency, alpha=0.25, color='blue')
    
    ax3.set_xticks(angles[:-1])
    ax3.set_xticklabels(tools)
    ax3.set_ylim([0, 1])
    ax3.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax3.set_yticklabels(['20%', '40%', '60%', '80%', '100%'])
    ax3.grid(True, alpha=0.3)
    
    # Next Steps and Recommendations
    ax4.set_title('Recommended Next Steps', fontsize=14, fontweight='bold')
    ax4.set_xlim([0, 10])
    ax4.set_ylim([0, 10])
    
    # Learning path recommendations
    next_steps = [
        (2, 8.5, "🔬 Research", "• Read robotics papers\\n• Follow latest developments\\n• Join robotics communities"),
        (8, 8.5, "🛠️ Practice", "• Build physical robots\\n• Implement algorithms\\n• Contribute to open source"),
        (2, 5.5, "📖 Deep Dive", "• Advanced control theory\\n• Machine learning\\n• Computer vision"),
        (8, 5.5, "🌐 Apply", "• Real-world projects\\n• Industry internships\\n• Research collaborations"),
        (5, 2.5, "🚀 Specialize", "• Choose focus area\\n• Advanced coursework\\n• Graduate research")
    ]
    
    for x, y, title, description in next_steps:
        # Create recommendation box
        bbox = Rectangle((x-1.2, y-1), 2.4, 2, facecolor='lightcyan', 
                        alpha=0.7, edgecolor='blue', linewidth=1)
        ax4.add_patch(bbox)
        ax4.text(x, y+0.3, title, ha='center', va='center', fontsize=12, fontweight='bold')
        ax4.text(x, y-0.3, description, ha='center', va='center', fontsize=9)
    
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return fig

def final_summary():
    """Print comprehensive course summary"""
    print("🎓 MODERN ROBOTICS LEARNING JOURNEY COMPLETE!")
    print("=" * 50)
    
    print("\\n📋 TOPICS COVERED:")
    topics = [
        "1. Configuration Space - Robot workspace and constraints",
        "2. Rigid Body Motions - SO(3) and SE(3) transformations", 
        "3. Forward Kinematics - Product of Exponentials method",
        "4. Jacobians - Velocity relationships and manipulability",
        "5. Trajectory Generation - Smooth motion planning",
        "6. Mecanum Wheels - Omnidirectional mobile robotics",
        "7. Feedforward Control - Open-loop trajectory tracking",
        "8. Feedback Control - Closed-loop error correction",
        "9. KUKA youBot Capstone - Complete system integration"
    ]
    
    for topic in topics:
        print(f"   ✅ {topic}")
    
    print("\\n🛠️ SKILLS DEVELOPED:")
    skills = [
        "Mathematical modeling using Lie groups and screw theory",
        "Geometric reasoning for 3D transformations",
        "Algorithm implementation in Python",
        "System integration and control design",
        "Interactive visualization and presentation",
        "Problem-solving with modern robotics tools"
    ]
    
    for skill in skills:
        print(f"   🎯 {skill}")
    
    print("\\n📚 KEY RESOURCES:")
    resources = [
        "Modern Robotics textbook by Lynch & Park",
        "modern_robotics Python library",
        "Interactive Jupyter notebook demonstrations",
        "Mathematical foundations of robotics",
        "Practical implementation examples"
    ]
    
    for resource in resources:
        print(f"   📖 {resource}")
    
    print("\\n🚀 FUTURE DIRECTIONS:")
    future = [
        "Explore advanced control techniques (MPC, adaptive control)",
        "Study machine learning applications in robotics",
        "Investigate multi-robot coordination and swarm intelligence", 
        "Delve into soft robotics and bio-inspired designs",
        "Apply knowledge to real-world robotics projects"
    ]
    
    for direction in future:
        print(f"   🌟 {direction}")
    
    print("\\n💡 KEY TAKEAWAYS:")
    takeaways = [
        "Modern robotics is built on solid mathematical foundations",
        "Geometric thinking is essential for 3D robot systems",
        "Integration of theory and practice leads to robust solutions",
        "Interactive tools enhance learning and understanding",
        "The field continues to evolve with exciting possibilities"
    ]
    
    for takeaway in takeaways:
        print(f"   ✨ {takeaway}")
    
    print("\\n🏆 CONGRATULATIONS!")
    print("You have successfully completed a comprehensive journey through")
    print("modern robotics, from fundamental mathematics to advanced applications.")
    print("This knowledge forms a solid foundation for continued learning and")
    print("innovation in the exciting field of robotics!")
    
    print("\\n" + "=" * 50)
    print("Happy Robot Building! 🤖✨")

# Generate learning summary
print("📚 Learning Journey Summary")
print("=" * 30)

fig = create_learning_summary()
final_summary()